In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
memory_data = pd.read_csv('./memory_anomaly_data2.csv')
memory_data['timestamp'] = pd.to_datetime(memory_data['timestamp'])

In [3]:
memory_data

,memory_usage,timestamp,anomaly
0,50.002197,2023-12-04 22:25:00+00:00,0
1,49.994604,2023-12-04 22:30:00+00:00,0
2,49.991114,2023-12-04 22:35:00+00:00,0
3,49.995935,2023-12-04 22:40:00+00:00,0
4,49.984534,2023-12-04 22:45:00+00:00,0
...,...,...,...
9990,67.120851,2024-01-08 14:55:00+00:00,0
9991,67.092148,2024-01-08 15:00:00+00:00,1
9992,67.100597,2024-01-08 15:05:00+00:00,1
9993,67.108008,2024-01-08 15:10:00+00:00,1


In [4]:
memory_data['memory_usage'] = memory_data['memory_usage']/100
memory_data

,memory_usage,timestamp,anomaly
0,0.500022,2023-12-04 22:25:00+00:00,0
1,0.499946,2023-12-04 22:30:00+00:00,0
2,0.499911,2023-12-04 22:35:00+00:00,0
3,0.499959,2023-12-04 22:40:00+00:00,0
4,0.499845,2023-12-04 22:45:00+00:00,0
...,...,...,...
9990,0.671209,2024-01-08 14:55:00+00:00,0
9991,0.670921,2024-01-08 15:00:00+00:00,1
9992,0.671006,2024-01-08 15:05:00+00:00,1
9993,0.671080,2024-01-08 15:10:00+00:00,1


In [5]:
num_lags = 2

#add lag
for i in range(1, num_lags+1):
    memory_data['lag_{}'.format(i)] = memory_data['memory_usage'].shift(i)
  
memory_data = memory_data.dropna()

In [6]:
memory_data

,memory_usage,timestamp,anomaly,lag_1,lag_2
2,0.499911,2023-12-04 22:35:00+00:00,0,0.499946,0.500022
3,0.499959,2023-12-04 22:40:00+00:00,0,0.499911,0.499946
4,0.499845,2023-12-04 22:45:00+00:00,0,0.499959,0.499911
5,0.500061,2023-12-04 22:50:00+00:00,0,0.499845,0.499959
6,0.500041,2023-12-04 22:55:00+00:00,0,0.500061,0.499845
...,...,...,...,...,...
9990,0.671209,2024-01-08 14:55:00+00:00,0,0.671005,0.671168
9991,0.670921,2024-01-08 15:00:00+00:00,1,0.671209,0.671005
9992,0.671006,2024-01-08 15:05:00+00:00,1,0.670921,0.671209
9993,0.671080,2024-01-08 15:10:00+00:00,1,0.671006,0.670921


In [9]:
X = memory_data[['memory_usage', 'lag_1', 'lag_2']]
y = memory_data['anomaly']

In [10]:
X_train = X[:int(0.7*len(X))]
X_test = X[int(0.7*len(X)):]
y_train = y[:int(0.7*len(y))]
y_test = y[int(0.7*len(y)):]

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

C:\Users\HP\AppData\Local\Temp\ipykernel_21300\3789776307.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train = y[:int(0.7*len(y))]
C:\Users\HP\AppData\Local\Temp\ipykernel_21300\3789776307.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test = y[int(0.7*len(y)):]


In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(3,), activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20


219/219 [==============================] - 2s 3ms/step - loss: 0.1280 - accuracy: 0.9887 - val_loss: 0.0678 - val_accuracy: 0.9880
Epoch 2/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0645 - accuracy: 0.9887 - val_loss: 0.0674 - val_accuracy: 0.9880
Epoch 3/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0643 - accuracy: 0.9887 - val_loss: 0.0663 - val_accuracy: 0.9880
Epoch 4/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0629 - accuracy: 0.9887 - val_loss: 0.0662 - val_accuracy: 0.9880
Epoch 5/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0628 - accuracy: 0.9887 - val_loss: 0.0650 - val_accuracy: 0.9880
Epoch 6/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0618 - accuracy: 0.9887 - val_loss: 0.0638 - val_accuracy: 0.9880
Epoch 7/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0606 - accuracy: 0.9887 - val_loss: 0.0626 - val_accuracy: 0.9880
Epoc

In [14]:
model.save('memory_model_2.h5')

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
